In [14]:
%reload_ext autoreload
%autoreload 2

In [15]:
import torch
import torchvision

In [ ]:
parse_args

In [8]:
a = { "a": 10, "b": "bb"}
b = {"a": 20, "b": "CC", "c": 20}

a.update(b)
a

{'a': 20, 'b': 'CC', 'c': 20}

In [9]:
a = {'b64_image': 'aa', 'dataset_mode': 'single', 'model': 'test'}

In [11]:
b = {'b64_image': 'aa', 'dataset_mode': 'unaligned', 'model': 'cycle_gan', 'dataroot': '/datasets/horse2zebra', 'checkpoints_dir': '/cache/ckpts', 'netD': 'basic', 'netG': 'resnet_9blocks', 'norm': 'instance', 'init_type': 'normal', 'init_gain': 0.02, 'no_dropout': True, 'direction': 'AtoB', 'num_threads': 4, 'batch_size': 4, 'max_dataset_size': 1024, 'no_flip': True, 'print_freq': 2, 'save_latest_freq': 20, 'continue_train': True, 'beta1': 0.5, 'lr': 0.002, 'lr_policy': 'linear', 'lr_decay_iters': 10, 'n_epochs': 120, 'gan_mode': 'lsgan', 'verbose': True}
b

{'b64_image': 'aa',
 'dataset_mode': 'unaligned',
 'model': 'cycle_gan',
 'dataroot': '/datasets/horse2zebra',
 'checkpoints_dir': '/cache/ckpts',
 'netD': 'basic',
 'netG': 'resnet_9blocks',
 'norm': 'instance',
 'init_type': 'normal',
 'init_gain': 0.02,
 'no_dropout': True,
 'direction': 'AtoB',
 'num_threads': 4,
 'batch_size': 4,
 'max_dataset_size': 1024,
 'no_flip': True,
 'print_freq': 2,
 'save_latest_freq': 20,
 'continue_train': True,
 'beta1': 0.5,
 'lr': 0.002,
 'lr_policy': 'linear',
 'lr_decay_iters': 10,
 'n_epochs': 120,
 'gan_mode': 'lsgan',
 'verbose': True}

In [12]:
a.update(b)
a

{'b64_image': 'aa',
 'dataset_mode': 'unaligned',
 'model': 'cycle_gan',
 'dataroot': '/datasets/horse2zebra',
 'checkpoints_dir': '/cache/ckpts',
 'netD': 'basic',
 'netG': 'resnet_9blocks',
 'norm': 'instance',
 'init_type': 'normal',
 'init_gain': 0.02,
 'no_dropout': True,
 'direction': 'AtoB',
 'num_threads': 4,
 'batch_size': 4,
 'max_dataset_size': 1024,
 'no_flip': True,
 'print_freq': 2,
 'save_latest_freq': 20,
 'continue_train': True,
 'beta1': 0.5,
 'lr': 0.002,
 'lr_policy': 'linear',
 'lr_decay_iters': 10,
 'n_epochs': 120,
 'gan_mode': 'lsgan',
 'verbose': True}